In [32]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
filepath='https://raw.githubusercontent.com/fedeamedeo/machine-learning-project/main/Data/'
interactions = pd.read_csv(filepath + 'interactions_train.csv')
filepath='https://raw.githubusercontent.com/fedeamedeo/machine-learning-project/main/'
items= pd.read_csv(filepath+'items_improved.csv')

In [34]:
# Load book metadata
items['content'] = items['Title'].fillna('') + ' ' + \
                   items['Author'].fillna('') + ' ' + \
                   items['Subjects'].fillna('') + ' ' + \
                   items['Publisher'].fillna('')

In [35]:
#minimal pre-processing
def clean_minimal(text):
    return re.sub(r"[^\w\s]", " ", str(text).lower())

In [36]:
items["clean_text_min"] = items["content"].fillna("").apply(clean_minimal)

In [37]:
# Add popularity boost
item_popularity = interactions['i'].value_counts(normalize=True)
items['popularity'] = items['i'].map(item_popularity).fillna(0)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Download French stopwords if not already
import nltk
nltk.download('stopwords')

# Load French stopwords as a list
french_stopwords = stopwords.words('french')
vectorizer = TfidfVectorizer(stop_words=french_stopwords, max_features=1000000, sublinear_tf=True, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(items['clean_text_min'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
from scipy.sparse import csr_matrix

# Merge interactions and items to match on 'i'
merged = interactions.merge(items[['i']], on='i')

# Create a user profile: mean of TF-IDF vectors of books they've interacted with
user_profiles = {}
for user_id, group in merged.groupby('u'):
    indices = group['i'].values
    profile = tfidf_matrix[indices].mean(axis=0).A1
    user_profiles[user_id] = profile

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

recommendations = []

for user_id, profile in user_profiles.items():
    # Reshape profile to ensure compatibility with tfidf_matrix
    sims = cosine_similarity(profile.reshape(1, -1), tfidf_matrix).flatten()
    boosted_sims = sims + 0.01 * items['popularity'].values
    top_indices = np.argsort(-boosted_sims)[:10]
    #top_indices = np.argsort(-sims)[:10]  # top 10
    book_ids = items.iloc[top_indices]['i'].tolist()
    recommendations.append((user_id, book_ids))

In [41]:
import csv

with open("tf_idf.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "recommendation"])
    for user_id, book_ids in recommendations:
        writer.writerow([user_id, " ".join(map(str, book_ids))])

In [42]:
from google.colab import files
files.download('tf_idf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>